In [ ]:
%%capture
# !git clone https://github.com/joerg84/Graph_Powered_ML_Workshop.git
# !rsync -av Graph_Powered_ML_Workshop/ ./ --exclude=.git
!pip3 install dgl-cu110 -f https://data.dgl.ai/wheels/repo.html
# !pip3 install dgl-cu102 -f https://data.dgl.ai/wheels/repo.html
!pip3 install numpy
!pip3 install torch
!pip3 install networkx
!pip3 install matplotlib
!pip3 install scikit-learn
!pip3 install chars2vec
!pip3 install keras
!pip3 install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir ./data/
!mkdir ./graph/
!mkdir ./pagerank/
!mkdir ./model

!cp ./drive/MyDrive/Colab\ Notebooks/han/data/fix_1k_labels.csv ./data/
!cp ./drive/MyDrive/Colab\ Notebooks/han/data/fix_3k_labels.csv ./data/
!cp ./drive/MyDrive/Colab\ Notebooks/han/data/fix_5k_labels.csv ./data/
!cp ./drive/MyDrive/Colab\ Notebooks/han/data/fix_1w_labels.csv ./data/
!cp ./drive/MyDrive/Colab\ Notebooks/han/data/fix_whois.csv ./data/
!cp ./drive/MyDrive/Colab\ Notebooks/han/model.py ./

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
load pirate data
'''
import re
from pprint import pprint

import re
from pprint import pprint

import chars2vec
import dgl
import numpy as np
import pandas as pd
import torch
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer


def get_raw_data(path):
    raw = pd.read_csv(path)
    reg_dic, dns_dic, email_dic = get_whois()
    print('Load Pirate Dataset: {}'.format(path))
    in_feat = {}
    website = {}
    third = {}
    w_ip = {}
    t_ip = {}
    cert = {}
    reg = {}
    dns = {}
    email = {}
    wi = 0
    ti = 0
    wpi = 0
    tpi = 0
    ci = 0
    ri = 0
    di = 0
    ei = 0
    edge_w_t = []
    edge_w_i = []
    edge_w_c = []
    edge_t_c = []
    edge_t_i = []
    edge_w_r = []
    edge_w_d = []
    edge_w_e = []
    labels = []

    '''
    raw data for website's ndoe embedding
    '''
    raw_reg = []
    raw_dns = []
    raw_email = []

    '''
    raw data for graph's node embedding
    '''
    node_domain = []
    node_third = []
    node_w_ip = []
    node_t_ip = []
    node_cert = []
    node_email = []
    node_reg = []
    node_dns = []


    for index, row in raw.iterrows():
        t = str(row['req']).split('/')[2].strip('www.')
        i = str(row['ip']).strip()
        w = str(row['website']).strip().split(':')[0]
        c = str(row['cert']).strip()
        if t != w:
            if t not in third:
                third[t] = ti
                node_third.append(t)
                ti += 1
            if i not in t_ip and i != 'nan':
                t_ip[i] = tpi
                node_t_ip.append(i)
                tpi += 1
        else:
            if i not in w_ip and i != 'nan':
                w_ip[i] = wpi
                node_w_ip.append(i)
                wpi += 1
        if w not in website:
            labels.append(int(row['label']))
            node_domain.append(w)
            raw_reg.append(reg_dic[w])
            raw_dns.append(dns_dic[w])
            raw_email.append(email_dic[w])
            website[w] = wi
            wi+=1
            if reg_dic[w] not in reg and reg_dic[w] != 'nan':
                reg[reg_dic[w]] = ri
                node_reg.append(reg_dic[w])
                ri += 1
            if dns_dic[w] != 'nan':
                for _ in dns_dic[w].split(','):
                    __ = _.strip().lower()
                    if __ not in dns:
                        dns[__] = di
                        node_dns.append(__)
                        di += 1
            if email_dic[w] not in email and email_dic[w] != 'nan':
                email[email_dic[w]] = ei
                node_email.append(email_dic[w])
                ei += 1
            in_feat[w] = [0, 0, 0]
        if c not in cert and c != 'nan':
            cert[c] = ci
            node_cert.append(c)
            ci += 1

        in_feat[w][0] += 1
        if t != w:
            edge_w_t.append(tuple([website[w], third[t]]))
            if i != 'nan':
              edge_t_i.append(tuple([third[t], t_ip[i]]))
              in_feat[w][1] += 1
            if c != 'nan':
                edge_t_c.append(tuple([third[t], cert[c]]))
        if t == w:
            if i != 'nan':
              edge_w_i.append(tuple([website[w], w_ip[i]]))
            if reg_dic[w] != 'nan':
                edge_w_r.append(tuple([website[w], reg[reg_dic[w]]]))
            if dns_dic[w] != 'nan':
                for _ in dns_dic[w].split(','):
                    __ = _.strip().lower()
                    edge_w_d.append(tuple([website[w], dns[__]]))
            if email_dic[w] != 'nan':
                edge_w_e.append(tuple([website[w], email[email_dic[w]]]))
            if c != 'nan':
                edge_w_c.append(tuple([website[w], cert[c]]))
                in_feat[w][2] += 1


    edge_w_i = set(edge_w_i)
    edge_w_t = set(edge_w_t)
    edge_w_c = set(edge_w_c)
    edge_t_c = set(edge_t_c)
    edge_t_i = set(edge_t_i)
    edge_w_r = set(edge_w_r)
    edge_w_d = set(edge_w_d)
    edge_w_e = set(edge_w_e)



    print('Node of Websites: ', len(website))
    print('Node of Third-party Services: ', len(third))
    print('Node of IP: ', len(w_ip))
    print('Node of Cert: ', len(cert))
    print('Node of Registrant: ',len(reg))
    print('Node of DNS: ', len(dns))
    print('Node of Email: ', len(email))
    print('Edge of Website to Third-party: ', len(edge_w_t))
    print('Edge of Webiste to IP: ', len(edge_w_i))
    print('Edge of Website to Certificate: ', len(edge_w_c))
    print('Edge of Website to Registrant: ', len(edge_w_r))
    print('Edge of Webiste to DNS Server: ', len(edge_w_d))
    print('Edge of Website to Email: ', len(edge_w_e))
    print('Edge of Third-party to Certificate: ', len(edge_t_c))
    print('Edge of Third-party to IP: ', len(edge_t_i))
    print('labels: ', len(labels))
    web_tp_src = []
    web_tp_dst = []
    web_ip_src = []
    web_ip_dst = []
    web_cert_src = []
    web_cert_dst = []
    third_cert_src = []
    third_cert_dst = []
    third_ip_src = []
    third_ip_dst = []
    web_reg_src = []
    web_reg_dst = []
    web_dns_src = []
    web_dns_dst = []
    web_email_src = []
    web_email_dst = []
    for item in edge_w_t:
        web_tp_src.append(item[0])
        web_tp_dst.append(item[1])
    for item in edge_w_i:
        web_ip_src.append(item[0])
        web_ip_dst.append(item[1])
    for item in edge_w_c:
        web_cert_src.append(item[0])
        web_cert_dst.append(item[1])
    for item in edge_t_c:
        third_cert_src.append(item[0])
        third_cert_dst.append(item[1])
    for item in edge_t_i:
        third_ip_src.append(item[0])
        third_ip_dst.append(item[1])
    for item in edge_w_r:
        web_reg_src.append(item[0])
        web_reg_dst.append(item[1])
    for item in edge_w_d:
        web_dns_src.append(item[0])
        web_dns_dst.append(item[1])
    for item in edge_w_e:
        web_email_src.append(item[0])
        web_email_dst.append(item[1])
    # hg = dgl.heterograph({
    #     ('web', 'wt', 'third'): (web_tp_src, web_tp_dst),
    #     ('third', 'tw', 'web'): (web_tp_dst, web_tp_src)},{'web':len(website),'third':len(third),'ip':len(ip),'cert':len(cert)})
    hg = dgl.heterograph({
        ('web', 'wt', 'third'): (web_tp_src, web_tp_dst),
        ('third', 'tw', 'web'): (web_tp_dst, web_tp_src),
        ('web', 'wi', 'ip'): (web_ip_src, web_ip_dst),
        ('ip', 'iw', 'web'): (web_ip_dst, web_ip_src),
        ('web', 'wc', 'cert'): (web_cert_src, web_cert_dst),
        ('cert', 'cw', 'web'): (web_cert_dst, web_cert_src),
        # ('third', 'tc', 'cert'): (third_cert_src, third_cert_dst),
        # ('cert', 'ct', 'third'): (third_cert_dst, third_cert_src),
        # ('third', 'ti', 'ip'): (third_ip_src, third_ip_dst),
        # ('ip', 'it', 'third'): (third_ip_dst, third_ip_src),
        ('web', 'wr', 'reg'): (web_reg_src, web_reg_dst),
        ('reg', 'rw', 'web'): (web_reg_dst, web_reg_src),
        ('web', 'we', 'email'): (web_email_src, web_email_dst),
        ('email', 'ew', 'web'): (web_email_dst, web_email_src),
        ('web', 'wd', 'dns'): (web_dns_src, web_dns_dst),
        ('dns', 'dw', 'web'): (web_dns_dst, web_dns_src)
    },
    # {'web':len(website),'ip':len(w_ip)}
    {'web':len(website),'third':len(third),'ip':len(w_ip),'dns':len(dns),'cert':len(cert),'reg':len(reg),'email':len(email)}
    )

    labels = torch.LongTensor(labels)

    '''
    one-hot features
    '''
    reg_oh = [[0 for _ in range(len(reg))] for __ in range(len(website))]
    email_oh = [[0 for _ in range(len(email))] for __ in range(len(website))]
    dns_oh = [[0 for _ in range(len(dns))] for __ in range(len(website))]


    for i in range(len(raw_reg)):
        if raw_reg[i]!='nan':
            reg_oh[i][reg[raw_reg[i]]] = 1
    for i in range(len(raw_email)):
        if raw_email[i] != 'nan':
            email_oh[i][email[raw_email[i]]] = 1
    for i in range(len(raw_dns)):
        for _ in raw_dns[i].split(','):
            __ = _.strip().lower()
            if __ != 'nan':
                dns_oh[i][dns[__]]=1

    reg_oh = np.array(reg_oh)
    email_oh = np.array(email_oh)
    dns_oh = np.array(dns_oh)


    in_features = []
    for item in node_domain:
        in_features.append(in_feat[item])

    data = {'labels': labels, 'node_domain': node_domain, 'raw_reg': raw_reg,
            'raw_dns': raw_reg, 'raw_email': raw_email,
            'hg': hg, 'in_features': np.array(in_features),'node_third':node_third,'node_w_ip':node_w_ip,'node_cert':node_cert,
            'node_reg':node_reg,'node_dns':node_dns,'node_email':node_email
            }
    import pickle
    pickle.dump(data, open('./graph/WTICRDE_1k.pkl', 'wb'))


def get_whois():
    raw = pd.read_csv('./data/fix_whois.csv')
    dic = {}
    website = {}
    email = {}
    dns = {}
    for web,x,d,e in zip(raw.website,raw.reg,raw.dns,raw.email):
        dns.setdefault(web,str(d))
        email.setdefault(web,str(e))
        tmp = str(x).strip().strip('.')
        if re.match(r'.*GoDaddy',tmp):
            dic.setdefault('GoDaddy.com, LLC',0)
            website.setdefault(web,'GoDaddy.com, LLC')
        elif re.match(r'.*NameSilo',tmp) or re.match(r'.*Namesilo',tmp):
            dic.setdefault('NameSilo, LLC',0)
            website.setdefault(web, 'NameSilo, LLC')
        elif re.match(r'.*Alibaba',tmp) or re.match(r'.*阿里巴巴',tmp) or re.match(r'.*阿里云',tmp):
            dic.setdefault('Alibaba Cloud Computing (Beijing) Co., Ltd',0)
            website.setdefault(web, 'Alibaba Cloud Computing (Beijing) Co., Ltd')
        elif re.match(r'.*新网',tmp) or re.match(r'Xin',tmp):
            dic.setdefault('Xin Net Technology Corporation', 0)
            website.setdefault(web, 'Xin Net Technology Corporation')
        elif re.match(r'NameCheap',tmp):
            dic.setdefault('NameCheap, Inc',0)
            website.setdefault(web, 'NameCheap, Inc')
        elif re.match(r'.*DropCatch',tmp):
            dic.setdefault('DropCatch.com, Inc',0)
            website.setdefault(web, 'DropCatch.com, Inc')
        elif re.match(r'.*Chengdu west',tmp) or re.match(r'.*Chengdu West',tmp) or re.match(r'.*成都西维',tmp):
            dic.setdefault('Chengdu West Dimension Digital Technology Co., Ltd',0)
            website.setdefault(web, 'Chengdu West Dimension Digital Technology Co., Ltd')
        elif re.match(r'.*Xiamen 35',tmp) or re.match(r'.*厦门三五',tmp):
            dic.setdefault('Xiamen 35.Com Technology Co., Ltd',0)
            website.setdefault(web, 'Xiamen 35.Com Technology Co., Ltd')
        elif re.match(r'.*1API',tmp):
            dic.setdefault('1API GmbH',0)
            website.setdefault(web, '1API GmbH')
        elif re.match(r'.*中央编办',tmp):
            dic.setdefault('China Gov',0)
            website.setdefault(web, 'China Gov')
        elif re.match(r'.*浙江贰',tmp):
            dic.setdefault('Zhejiang ErEr Technology Co., Ltd',0)
            website.setdefault(web, 'Zhejiang ErEr Technology Co., Ltd')
        elif re.match(r'.*Dynadot',tmp):
            dic.setdefault('Dynadot, LLC',0)
            website.setdefault(web, 'Dynadot, LLC')
        elif re.match(r'.*广州云',tmp):
            dic.setdefault('Guangzhou Cloud Technology Co., Ltd',0)
            website.setdefault(web, 'Guangzhou Cloud Technology Co., Ltd')
        elif re.match(r'.*厦门纳网',tmp):
            dic.setdefault('Xiamen NaNetwork Technology Co., Ltd',0)
            website.setdefault(web, 'Xiamen NaNetwork Technology Co., Ltd')
        elif re.match(r'.*厦门易名',tmp):
            dic.setdefault('Xiamen Easy Name Technology Co., Ltd',0)
            website.setdefault(web, 'Xiamen Easy Name Technology Co., Ltd')
        elif re.match(r'.*四川域趣',tmp):
            dic.setdefault('Sichuan Fun Technology Co., Ltd',0)
            website.setdefault(web, 'Sichuan Fun Technology Co., Ltd')
        elif re.match(r'.*成都飞数',tmp):
            dic.setdefault('Chengdu Fly Digital Technology Co., Ltd',0)
            website.setdefault(web, 'Chengdu Fly Digital Technology Co., Ltd')
        elif re.match(r'.*北京光速',tmp):
            dic.setdefault('Beijing LightSpeed Technology Co., Ltd',0)
            website.setdefault(web, 'Beijing LightSpeed Technology Co., Ltd')
        elif re.match(r'.*上海福虎',tmp):
            dic.setdefault('Shanghai Tiger Technology Co., Ltd',0)
            website.setdefault(web, 'Shanghai Tiger Technology Co., Ltd')
        elif re.match(r'.*广东时代',tmp):
            dic.setdefault('Guangdong TimeInternet Technology Co., Ltd',0)
            website.setdefault(web, 'Guangdong TimeInternet Technology Co., Ltd')
        elif re.match(r'.*成都世纪',tmp):
            dic.setdefault('Chengdu Century Oriental Network Communication Co., Ltd',0)
            website.setdefault(web, 'Chengdu Century Oriental Network Communication Co., Ltd')
        elif re.match(r'.*北京国旭',tmp):
            dic.setdefault('Beijing Guoxu Network Technology Co., Ltd', 0)
            website.setdefault(web, 'Beijing Guoxu Network Technology Co., Ltd')
        elif re.match(r'.*北京中科',tmp):
            dic.setdefault('Beijing Sanfront Information Technology Co., Ltd', 0)
            website.setdefault(web, 'Beijing Sanfront Information Technology Co., Ltd')
        elif re.match(r'.*北京东方',tmp):
            dic.setdefault('Beijing East Network Technology Co., Ltd', 0)
            website.setdefault(web, 'Beijing East Network Technology Co., Ltd')
        elif re.match(r'.*商中在线',tmp):
            dic.setdefault('Shangzhong Online Technology Co., Ltd', 0)
            website.setdefault(web, 'Shangzhong Online Technology Co., Ltd')
        elif re.match(r'.*中企动力',tmp):
            dic.setdefault('Zhong Qi Dong Li Technology Co., Ltd', 0)
            website.setdefault(web, 'Zhong Qi Dong Li Technology Co., Ltd')
        elif re.match(r'.*厦门市中资',tmp):
            dic.setdefault('Xiamen ChinaSource Internet Service Co., Ltd', 0)
            website.setdefault(web, 'Xiamen ChinaSource Internet Service Co., Ltd')
        elif re.match(r'.*广东互易',tmp):
            dic.setdefault('Guangdong HuYi Network Technology Co., Ltd', 0)
            website.setdefault(web, 'Guangdong HuYi Network Technology Co., Ltd')
        elif re.match(r'.*遵义中域',tmp):
            dic.setdefault('Zunyi Mid Technology Co., Ltd', 0)
            website.setdefault(web, 'Zunyi Mid Technology Co., Ltd')
        elif re.match(r'.*北京首信网',tmp):
            dic.setdefault('Beijing Trust Network Technology Co., Ltd', 0)
            website.setdefault(web, 'Beijing Trust Network Technology Co., Ltd')
        elif re.match(r'.*赛尔网络',tmp):
            dic.setdefault('Saier Network Technology Co., Ltd', 0)
            website.setdefault(web, 'Saier Network Technology Co., Ltd')
        elif re.match(r'.*遵义中域',tmp):
            dic.setdefault('Zunyi Mid Technology Co., Ltd', 0)
            website.setdefault(web, 'Zunyi Mid Technology Co., Ltd')
        elif re.match(r'.*烟台帝思普',tmp):
            dic.setdefault('Yantai DiSiPu Technology Co., Ltd', 0)
            website.setdefault(web, 'Yantai DiSiPu Technology Co., Ltd')
        elif re.match(r'.*北京万维',tmp):
            dic.setdefault('Beijing WanWei Technology Co., Ltd', 0)
            website.setdefault(web, 'Beijing WanWei Technology Co., Ltd')
        elif re.match(r'.*江苏邦宁',tmp):
            dic.setdefault('Jiangsu Bangning Science and technology Co. Ltd', 0)
            website.setdefault(web, 'Jiangsu Bangning Science and technology Co. Ltd')
        elif re.match(r'.*北京神州',tmp):
            dic.setdefault('Beijing ShenZhou technology Co. Ltd', 0)
            website.setdefault(web, 'Beijing ShenZhou technology Co. Ltd')
        elif re.match(r'.*北京神州',tmp):
            dic.setdefault('Beijing ShenZhou technology Co. Ltd', 0)
            website.setdefault(web, 'Beijing ShenZhou technology Co. Ltd')
        elif re.match(r'.*上海美橙',tmp):
            dic.setdefault('Shanghai Meicheng Technology Information Development Co., Ltd', 0)
            website.setdefault(web, 'Shanghai Meicheng Technology Information Development Co., Ltd')
        elif re.match(r'.*广东金万',tmp):
            dic.setdefault('Guangdong JinWanBang Technology Investment Co., Ltd', 0)
            website.setdefault(web, 'Guangdong JinWanBang Technology Investment Co., Ltd')
        elif re.match(r'.*易介集团',tmp):
            dic.setdefault('YiJie Beijing technology Co. Ltd', 0)
            website.setdefault(web, 'YiJie Beijing technology Co. Ltd')
        elif re.match(r'.*佛山市亿动',tmp):
            dic.setdefault('Foshan YiDong Network Co., LTD', 0)
            website.setdefault(web, 'Foshan YiDong Network Co., LTD')
        elif re.match(r'.*山东开创',tmp):
            dic.setdefault('Shandong KaiChuang technology Co. Ltd', 0)
            website.setdefault(web, 'Shandong KaiChuang technology Co. Ltd')
        elif re.match(r'.*泛亚信息',tmp):
            dic.setdefault('FanYa Jiangsu technology Co. Ltd', 0)
            website.setdefault(web, 'FanYa Jiangsu technology Co. Ltd')
        elif re.match(r'.*杭州电商',tmp):
            dic.setdefault('Hangzhou Dianshang Internet Technology Co., LTD', 0)
            website.setdefault(web, 'Hangzhou Dianshang Internet Technology Co., LTD')
        elif re.match(r'.*深圳市万维',tmp):
            dic.setdefault('Shenzhen WanWei technology Co. Ltd', 0)
            website.setdefault(web, 'Shenzhen WanWei technology Co. Ltd')
        elif re.match(r'.*天津追日',tmp):
            dic.setdefault('Tianjin Zhuiri Science and Technology Development Co Ltd', 0)
            website.setdefault(web, 'Tianjin Zhuiri Science and Technology Development Co Ltd')
        elif re.match(r'.*北京宏网',tmp):
            dic.setdefault('Beijing HongWang technology Co. Ltd', 0)
            website.setdefault(web, 'Beijing HongWang technology Co. Ltd')
        elif re.match(r'.*昆明乐网',tmp):
            dic.setdefault('Kunming LeWang technology Co. Ltd', 0)
            website.setdefault(web, 'Kunming LeWang technology Co. Ltd')
        elif re.match(r'.*上海热线',tmp):
            dic.setdefault('Shanghai ReXian technology Co. Ltd', 0)
            website.setdefault(web, 'Shanghai ReXian technology Co. Ltd')
        elif re.match(r'.*互联网域名',tmp):
            dic.setdefault('Internet Domain Name System Beijing Engineering Research Center LLC (ZDNS)', 0)
            website.setdefault(web, 'Internet Domain Name System Beijing Engineering Research Center LLC (ZDNS)')
        elif re.match(r'.*广东耐思',tmp):
            dic.setdefault('SGuangdong Nicenic Technology Co., Ltd. dba NiceNIC', 0)
            website.setdefault(web, 'Guangdong Nicenic Technology Co., Ltd. dba NiceNIC')
        elif re.match(r'.*重庆智佳',tmp):
            dic.setdefault('Chongqing ZhiJia Technology Co., Ltd', 0)
            website.setdefault(web, 'Chongqing ZhiJia Technology Co., Ltd')
        else:
            dic.setdefault(tmp,0)
            website.setdefault(web, tmp)
    return website,dns,email

def domain_to_index(domain, length):
    """

    :param domain: the domain data
    :param length: the csv data's length
    :return: index_vector of the domain
    """

    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(domain)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(domain)

    # padding
    data = pad_sequences(sequences, maxlen=30)
    # print('Found %s unique tokens.' % len(word_index))
    # print(data)
    print('Start Transform Character to index')
    return np.array(data, dtype="float32").reshape(length, 30), word_index

def load_pirate(path):
    return get_raw_data(path)


def load_pkl(dim,path):
    print('Load From PKL:',path)
    import pickle
    data = pickle.load(open(path, 'rb'))
    hg = data['hg']

    labels = data['labels']

    '''
    character-embedding features
    '''
    # features, _ = domain_to_index(raw_domain, len(raw_domain))
    c2v_model = \
        chars2vec.load_model(dim)
    website_features = c2v_model.vectorize_words(data['node_domain'])
    web_reg_features = c2v_model.vectorize_words(data['raw_reg'])
    web_dns_features = c2v_model.vectorize_words(data['raw_dns'])
    web_email_features = c2v_model.vectorize_words(data['raw_email'])

    third_features = c2v_model.vectorize_words(data['node_third'])
    ip_features = c2v_model.vectorize_words(data['node_w_ip'])
    cert_features = c2v_model.vectorize_words(data['node_cert'])
    reg_features = c2v_model.vectorize_words(data['node_reg'])
    dns_features = c2v_model.vectorize_words(data['node_dns'])
    email_features = c2v_model.vectorize_words(data['node_email'])

    features = {
        'web': torch.FloatTensor(website_features),
        'third': torch.FloatTensor(third_features),
        'ip': torch.FloatTensor(ip_features),
        'cert': torch.FloatTensor(cert_features),
        'email': torch.FloatTensor(email_features),
        'dns':torch.FloatTensor(dns_features),
        'reg':torch.FloatTensor(reg_features),
    }

    pprint({
        'Web Features:':features['web'].shape,
        'Third Features:': features['third'].shape,
        'IP Features:': features['ip'].shape,
        'Cert Features:': features['cert'].shape,
        'Reg Features:': features['reg'].shape,
        'DNS Features:': features['dns'].shape,
        'Email Features:': features['email'].shape
    })

    in_features = data['in_features']
    num_classes = 2


    # plot(features,labels)

    return hg, features, labels, num_classes

load_pirate('./data/fix_1k_labels.csv')
# load_pkl('eng_50','./graph/WTICRDE_1w.pkl')

In [ ]:
'''
utils.py
'''
import datetime
import dgl
import errno
import numpy as np
import os
import pickle
import random
import torch

from pprint import pprint
from scipy import sparse
from scipy import io as sio


def set_random_seed(seed=0):
    """Set random seed.
    Parameters
    ----------
    seed : int
        Random seed to use
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

def mkdir_p(path, log=True):
    """Create a directory for the specified path.
    Parameters
    ----------
    path : str
        Path name
    log : bool
        Whether to print result for directory creation
    """
    try:
        os.makedirs(path)
        if log:
            print('Created directory {}'.format(path))
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path) and log:
            print('Directory {} already exists.'.format(path))
        else:
            raise

def get_date_postfix():
    """Get a date based postfix for directory name.
    Returns
    -------
    post_fix : str
    """
    dt = datetime.datetime.now()
    post_fix = '{}_{:02d}-{:02d}-{:02d}'.format(
        dt.date(), dt.hour, dt.minute, dt.second)

    return post_fix

def setup_log_dir(args, sampling=False):
    """Name and create directory for logging.
    Parameters
    ----------
    args         Configuration
    Returns
    -------
    log_dir : str
        Path for logging directory
    sampling : bool
        Whether we are using sampling based training
    """
    date_postfix = get_date_postfix()
    log_dir = os.path.join(
        args['log_dir'],
        '{}_{}'.format(args['dataset'], date_postfix))

    if sampling:
        log_dir = log_dir + '_sampling'

    mkdir_p(log_dir)
    return log_dir

# The configuration below is from the paper.
default_configure = {
    'lr': 0.005,             # Learning rate
    'num_heads': [8],        # Number of attention heads for node-level attention
    'hidden_units': 8,
    'dropout': 0.6,
    'weight_decay': 0.001,
    'patience': 100
}

sampling_configure = {
    'batch_size': 20
}

def setup(args):
    args.update(default_configure)
    set_random_seed(args['seed'])
    args['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
    return args

def setup_for_sampling(args):
    args.update(default_configure)
    args.update(sampling_configure)
    set_random_seed()
    args['device'] = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    return args

def get_binary_mask(total_size, indices):
    mask = torch.zeros(total_size)
    mask[indices] = 1
    return mask.byte()


def load_data(dim,dataset, remove_self_loop=False):
    print('Load Dataset:',dataset)
    if dataset == 'Pirate1RDE':
        return load_pkl(dim,'graph/WTICRDE_1k.pkl')
    elif dataset == 'Pirate3':
        return load_pkl(dim,'graph/WTIC_3k.pkl')
    elif dataset == 'Pirate10':
        return load_pkl(dim,'graph/WTIC_1w.pkl')
    elif dataset == 'Pirate10RDE':
        return load_pkl(dim,'graph/WTICRDE_1w.pkl')
    elif dataset == 'Pirate5RDE':
        return load_pkl(dim,'graph/WTICRDE_5k.pkl')
    elif dataset == 'Pirate3RDE':
        return load_pkl(dim,'graph/WTICRDE_3k.pkl')
    elif dataset == 'Pirate10RE':
        return load_pkl(dim,'graph/WTICRE_1w.pkl')
    elif dataset == 'Pirate10RD':
        return load_pkl(dim,'graph/WTICRD_1w.pkl')
    elif dataset == 'Pirate10DE':
        return load_pkl(dim,'graph/WTICDE_1w.pkl')
    elif dataset == 'Pirate10R':
        return load_pkl(dim,'graph/WTICR_1w.pkl')
    elif dataset == 'Pirate10WTID':
        return load_pkl(dim,'graph/WTID_1w.pkl')
    elif dataset == 'Pirate10WT':
        return load_pkl(dim,'graph/WT_1w.pkl')
    elif dataset == 'Pirate10WI':
        return load_pkl(dim,'graph/WI_1w.pkl')
    elif dataset == 'Pirate10WD':
        return load_pkl(dim,'graph/WD_1w.pkl')
    elif dataset == 'Pirate10WC':
        return load_pkl(dim,'graph/WC_1w.pkl')
    elif dataset == 'Pirate10WE':
        return load_pkl(dim,'graph/WE_1w.pkl')
    elif dataset == 'Pirate10WR':
        return load_pkl(dim,'graph/WR_1w.pkl')
    elif dataset == 'Pirate10NoC':
        return load_pkl(dim,'graph/WTIRDE_1w.pkl')
    else:
        return NotImplementedError('Unsupported dataset {}'.format(dataset))

class EarlyStopping(object):
    def __init__(self, patience=10):
        dt = datetime.datetime.now()
        self.filename = './model/early_stop_{}_{:02d}-{:02d}-{:02d}.pth'.format(
            dt.date(), dt.hour, dt.minute, dt.second)
        self.patience = patience
        self.counter = 0
        self.best_acc = None
        self.best_loss = None
        self.early_stop = False

    def step(self, loss, acc, model):
        if self.best_loss is None:
            self.best_acc = acc
            self.best_loss = loss
            self.save_checkpoint(model)
        elif (loss > self.best_loss) and (acc < self.best_acc):
            self.counter += 1
            # print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if (loss <= self.best_loss) and (acc >= self.best_acc):
                self.save_checkpoint(model)
            self.best_loss = np.min((loss, self.best_loss))
            self.best_acc = np.max((acc, self.best_acc))
            self.counter = 0
        return self.early_stop

    def save_checkpoint(self, model):
        """Saves model when validation loss decreases."""
        torch.save(model.state_dict(), self.filename)

    def load_checkpoint(self, model):
        """Load the latest checkpoint."""
        model.load_state_dict(torch.load(self.filename))


def plot(features,pred,labels):
    import torch
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    def PCA(data, k=2):
        X = data
        X_mean = torch.mean(X, 0)
        X = X - X_mean.expand_as(X)
        # SVD
        U, S, V = torch.svd(torch.t(X))
        return torch.mm(X, U[:, :k])

    X = features
    y = []
    for i in range(len(labels)):
      if pred[i]==labels[i] and pred[i]==1:
        y.append(1)
      elif pred[i]==labels[i] and pred[i]==0:
        y.append(2)
      elif pred[i]!=labels[i] and pred[i]==0:
        y.append(-1)
      else:
        y.append(-2)
    y = torch.tensor(y)
    # y = labels
    X_pca = PCA(X)
    pca = X_pca.numpy()
    #
    plt.figure()
    color = cm.rainbow(np.linspace(0, 1, 4))
    plt.scatter(pca[y == 1,0], pca[y == 1,1],s=5, label='True Positive', color=color[0],marker='o')
    plt.scatter(pca[y == 2,0], pca[y == 2,1],s=5, label='True Negative', color=color[1],marker='o')
    plt.scatter(pca[y == -1,0], pca[y == -1,1],s=100, label='False Positive', color=color[2],marker='*')
    plt.scatter(pca[y == -2,0], pca[y == -2,1],s=100, label='False Negative', color=color[3],marker='*')

    plt.legend()
    plt.title('PCA of Pirate 1W dataset')
    plt.show(): dict


# load_data('eng_50','Pirate10RDE')

In [ ]:
'''
Focal Loss.py
'''
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.cuda.amp as amp


##
# version 1: use torch.autograd
class FocalLossV1(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean',):
        super(FocalLossV1, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.crit = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, label):
        '''
        Usage is same as nn.BCEWithLogits:
            >>> criteria = FocalLossV1()
            >>> logits = torch.randn(8, 19, 384, 384)
            >>> lbs = torch.randint(0, 2, (8, 19, 384, 384)).float()
            >>> loss = criteria(logits, lbs)
        '''
        probs = torch.sigmoid(logits)
        coeff = torch.abs(label - probs).pow(self.gamma).neg()
        log_probs = torch.where(logits >= 0,
                F.softplus(logits, -1, 50),
                logits - F.softplus(logits, 1, 50))
        log_1_probs = torch.where(logits >= 0,
                -logits + F.softplus(logits, -1, 50),
                -F.softplus(logits, 1, 50))
        loss = label * self.alpha * log_probs + (1. - label) * (1. - self.alpha) * log_1_probs
        loss = loss * coeff

        if self.reduction == 'mean':
            loss = loss.mean()
        if self.reduction == 'sum':
            loss = loss.sum()
        return loss


##
# version 2: user derived grad computation
class FocalSigmoidLossFuncV2(torch.autograd.Function):
    '''
    compute backward directly for better numeric stability
    '''
    @staticmethod
    @amp.custom_fwd(cast_inputs=torch.float32)
    def forward(ctx, logits, label, alpha, gamma):
        #  logits = logits.float()

        probs = torch.sigmoid(logits)
        coeff = (label - probs).abs_().pow_(gamma).neg_()
        log_probs = torch.where(logits >= 0,
                F.softplus(logits, -1, 50),
                logits - F.softplus(logits, 1, 50))
        log_1_probs = torch.where(logits >= 0,
                -logits + F.softplus(logits, -1, 50),
                -F.softplus(logits, 1, 50))
        ce_term1 = log_probs.mul_(label).mul_(alpha)
        ce_term2 = log_1_probs.mul_(1. - label).mul_(1. - alpha)
        ce = ce_term1.add_(ce_term2)
        loss = ce * coeff

        ctx.vars = (coeff, probs, ce, label, gamma, alpha)

        return loss

    @staticmethod
    @amp.custom_bwd
    def backward(ctx, grad_output):
        '''
        compute gradient of focal loss
        '''
        (coeff, probs, ce, label, gamma, alpha) = ctx.vars

        d_coeff = (label - probs).abs_().pow_(gamma - 1.).mul_(gamma)
        d_coeff.mul_(probs).mul_(1. - probs)
        d_coeff = torch.where(label < probs, d_coeff.neg(), d_coeff)
        term1 = d_coeff.mul_(ce)

        d_ce = label * alpha
        d_ce.sub_(probs.mul_((label * alpha).mul_(2).add_(1).sub_(label).sub_(alpha)))
        term2 = d_ce.mul(coeff)

        grads = term1.add_(term2)
        grads.mul_(grad_output)

        return grads, None, None, None


class FocalLossV2(nn.Module):

    def __init__(self,
                 alpha=0.25,
                 gamma=2,
                 reduction='mean'):
        super(FocalLossV2, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, label):
        '''
        Usage is same as nn.BCEWithLogits:
            >>> criteria = FocalLossV2()
            >>> logits = torch.randn(8, 19, 384, 384)
            >>> lbs = torch.randint(0, 2, (8, 19, 384, 384)).float()
            >>> loss = criteria(logits, lbs)
        '''
        loss = FocalSigmoidLossFuncV2.apply(logits, label, self.alpha, self.gamma)
        if self.reduction == 'mean':
            loss = loss.mean(dim=1)
        if self.reduction == 'sum':
            loss = loss.sum(dim=1)
        return loss


In [ ]:
'''
Renode.py
'''
import math

import torch
import torch.nn.functional as F
import numpy as np

def page_rank(args,adj,filename):
    print('Calculate Pagerank')
    pr_prob = 1 - args['pagerank_prob']
    A = adj.to_dense()
    A_hat = A.to(args['device']) + torch.eye(A.size(0)).to(args['device'])  # add self-loop
    D = torch.diag(torch.sum(A_hat, 1))
    D = D.inverse().sqrt()
    A_hat = torch.mm(torch.mm(D, A_hat), D)
    Pi = pr_prob * ((torch.eye(A.size(0)).to(args['device']) - (1 - pr_prob) * A_hat).inverse())
    Pi = Pi.cpu()
    torch.save(Pi, filename)
    return Pi.to(args['device'])

def get_gpr(Pi,train_node,labels):
    gpr_matrix = []  # the class-level influence distribution
    for iter_c in range(2):
        iter_Pi = Pi[torch.tensor(train_node[iter_c]).long()]
        iter_gpr = torch.mean(iter_Pi, dim=0).squeeze()
        gpr_matrix.append(iter_gpr)
    '''
        gpr_matrix = [tensor[0],tensor[1]]
        tensor[0] = []
        temp_gpr = []
    '''

    temp_gpr = torch.stack(gpr_matrix, dim=0)
    temp_gpr = temp_gpr.transpose(0, 1)
    gpr = temp_gpr

    return gpr
    # rn_weight = get_renode_weight(gpr,)  # ReNode Weight


def get_renode_weight(args,Pi,gpr,labels,train_mask):
    ppr_matrix = Pi  # personlized pagerank
    gpr_matrix = torch.tensor(gpr).float()  # class-accumulated personlized pagerank

    base_w = args['rn_base_weight']
    scale_w = args['rn_scale_weight']
    nnode = ppr_matrix.size(0)
    # unlabel_mask = data.train_mask.int().ne(1)  # unlabled node

    # computing the Totoro values for labeled nodes
    '''
        gpr_sum 对每个节点各label维度上与pagerank计算出的权重进行求和
        gpr_rn 转置后减gpr_matrix
        rn_matrix pagerank矩阵与每个节点计算出的类别权重进行相乘
    '''
    gpr_sum = torch.sum(gpr_matrix, dim=1)
    gpr_rn = gpr_sum.unsqueeze(1) - gpr_matrix
    rn_matrix = torch.mm(ppr_matrix, gpr_rn)

    label_matrix = F.one_hot(labels, gpr_matrix.size(1)).float()
    # label_matrix[unlabel_mask] = 0


    '''
    rn_matrix 根据rn矩阵及label标签得到每个节点的rn值
    '''
    rn_matrix = torch.sum(rn_matrix * label_matrix, dim=1)
    # rn_matrix[unlabel_mask] = rn_matrix.max() + 99  # exclude the influence of unlabeled node


    '''
        根据节点rn值从小到大排序
    '''
    # computing the ReNode Weight
    train_size = torch.sum(train_mask.int()).item()
    totoro_list = rn_matrix.tolist()
    id2totoro = {i: totoro_list[i] for i in range(len(totoro_list))}

    sorted_totoro = sorted(id2totoro.items(), key=lambda x: x[1], reverse=False)

    id2rank = {sorted_totoro[i][0]: i for i in range(nnode)}

    totoro_rank = [id2rank[i] for i in range(nnode)]


    rn_weight = [(base_w + 0.5 * scale_w * (1 + math.cos(x * 1.0 * math.pi / (train_size - 1)))) for x in totoro_rank]
    rn_weight = torch.from_numpy(np.array(rn_weight)).type(torch.FloatTensor).to(args['device'])
    rn_weight = rn_weight * train_mask.float()

    # print(rn_weight.size())
    # print('rn size: ',rn_weight.size())
    return rn_weight


'''
Get PageRank
'''

def hg2Pi(args,g):
    # metapath = ['wi']
    metapath = ['wt', 'wi', 'wd','wc','we','wr']
    # metapath = ['wt', 'wi', 'wd','we','wr']
    Pi = {}
    for item in metapath:
        if os.path.exists('./pagerank/'+args['dataset'] + item + '.pt'):
            Pi[item] = torch.load('./pagerank/'+args['dataset'] + item + '.pt').to(args['device'])
        else:
            meta_g = dgl.metapath_reachable_graph(g, [item, item[::-1]])
            Pi[item] = page_rank(args, meta_g.adjacency_matrix(),'./pagerank/'+args['dataset'] + item + '.pt')

    return Pi

'''
Get ReNode Weight
'''

def hg2rn(args,Pi,att_score, train_node, train_mask, labels):
    # metapath = ['wi']
    metapath = ['wt','wi','wd','wc','we','wr']
    # metapath = ['wt','wi','wd','we','wr']
    weight = {}
    for item in metapath:
        pi = Pi[item]*att_score[item]
        gpr = get_gpr(pi, train_node, labels)
        weight[item] = get_renode_weight(args, pi, gpr, labels, train_mask)

    # for k,v in weight.items():
    #     print(k,v)
    res = torch.zeros(len(labels)).to(args['device'])
    for _ in weight.values():
        res = torch.add(res,_)
    return res

In [ ]:
'''
HGT train.py
'''
from pprint import pprint

import torch
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, precision_score, \
    recall_score, precision_recall_fscore_support

from model import *
import numpy as np
from scipy.sparse import coo_matrix



def score(logits, labels):
    _, indices = torch.max(logits, dim=1)
    prediction = indices.long().cpu().numpy()
    labels = labels.cpu().numpy()

    accuracy = (prediction == labels).sum() / len(prediction)
    micro_f1 = f1_score(labels, prediction, average='micro')
    macro_f1 = f1_score(labels, prediction, average='macro')

    return accuracy, micro_f1, macro_f1


def hgt_evaluate(model, g, t, labels, mask, loss_func):
    model.eval()
    with torch.no_grad():
        logits = model(g, t)
    loss = loss_func(logits[mask], F.one_hot(labels[mask], num_classes=2).float())
    accuracy, micro_f1, macro_f1 = score(logits[mask], labels[mask])
    # indices = indices.cpu().numpy()
    # labels = labels.cpu().numpy()
    _, indices = torch.max(logits[mask], dim=1)
    val_precision = precision_score(labels[mask].cpu().numpy(), indices.cpu().numpy(), average='macro')
    val_recall = recall_score(labels[mask].cpu().numpy(), indices.cpu().numpy(), average='macro')

    return loss, accuracy, micro_f1, macro_f1, val_precision, val_recall


def hgt_test(model, g, t, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, t)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        '''
        Plot output feature
        '''
        # feature_output1 = model.featuremap[mask].cpu()
        # plot(feature_output1,indices.cpu(),labels.cpu())

        indices = indices.cpu().numpy()
        labels = labels.cpu().numpy()
        p,r,f1,s = precision_recall_fscore_support(labels,indices)

        print(classification_report(labels, indices, digits=4))
        print(confusion_matrix(labels, indices))
    return p[1],r[1],f1[1]


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp

def att_score(g,model):
    '''
        Using subgraph to calculate the website attention score matrix
    '''
    meta = model.meta
    att_score = {}
    for k, v in meta.items():
        row, col = v['edges']
        # row = row.cpu().numpy()
        # col = col.cpu().numpy()
        tmp = v['att_score']

        indices = torch.stack((row,col),0)
        if k == 'wt':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('third')])
        elif k == 'tw':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('third'), g.num_nodes('web')])
        elif k == 'iw':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('ip'), g.num_nodes('web')])
        elif k == 'wi':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('ip')])
        elif k == 'wd':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('dns')])
        elif k == 'dw':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('dns'), g.num_nodes('web')])
        elif k == 'ew':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('email'), g.num_nodes('web')])
        elif k == 'we':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('email')])
        elif k == 'wr':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('reg')])
        elif k == 'rw':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('reg'), g.num_nodes('web')])
        elif k == 'wc':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('web'), g.num_nodes('cert')])
        elif k == 'cw':
            att_score[k] = torch.sparse_coo_tensor(indices,tmp,[g.num_nodes('cert'), g.num_nodes('web')])
    # for k, v in att_score.items():
    #     print(v.shape)
    re_att = {}
    re_att['wt'] = torch.mm(att_score['wt'].to_dense(), att_score['tw'].to_dense())
    # print(re_att)
    re_att['wi'] = torch.mm(att_score['wi'].to_dense(), att_score['iw'].to_dense())
    re_att['wd'] = torch.mm(att_score['wd'].to_dense(), att_score['dw'].to_dense())
    re_att['wr'] = torch.mm(att_score['wr'].to_dense(), att_score['rw'].to_dense())
    re_att['wc'] = torch.mm(att_score['wc'].to_dense(), att_score['cw'].to_dense())
    re_att['we'] = torch.mm(att_score['we'].to_dense(), att_score['ew'].to_dense())
    return re_att

def get_train_node(train_idx,labels):
    train_node=[[],[]]
    for i in train_idx:
        if labels[i]==1:
            train_node[1].append(i)
        else:
            train_node[0].append(i)
    return train_node


def get_binary_mask(total_size, indices):
    mask = torch.zeros(total_size)
    mask[indices] = 1
    return mask.byte()


def shuffule(hg,labels):
    float_mask = np.random.permutation(np.linspace(0, 1, len(labels)))
    train_idx = np.where(float_mask <= 0.6)[0]
    val_idx = np.where((float_mask > 0.6) & (float_mask <= 0.8))[0]
    test_idx = np.where(float_mask > 0.8)[0]

    num_nodes = hg.number_of_nodes('web')
    train_mask = get_binary_mask(num_nodes, train_idx)
    val_mask = get_binary_mask(num_nodes, val_idx)
    test_mask = get_binary_mask(num_nodes, test_idx)

    print('dataset loaded')
    pprint({
        'dataset': 'Pirate',
        'train': train_mask.sum().item() / num_nodes,
        'val': val_mask.sum().item() / num_nodes,
        'test': test_mask.sum().item() / num_nodes
    })
    return train_idx, val_idx, test_idx, train_mask, val_mask, test_mask


def hgt_batch(args,model, g,train_idx, train_mask, val_mask, test_mask, labels, Pi,train_node):
    stopper = EarlyStopping(patience=args['patience'])
    if args['loss']=='BCE':
        loss_fcn = torch.nn.BCEWithLogitsLoss()
    if args['loss'] =='BCERN':
        loss_fcn = torch.nn.BCEWithLogitsLoss(reduce=False)
    if args['loss']=='Focal' or args['loss'] =='RN':
        loss_fcn = FocalLossV2()
    # loss_fcn = torch.nn.CrossEntropyLoss()

    optimizer = torch.optim.AdamW(model.parameters())

    for epoch in range(args['num_epochs']):
        model.train()
        logits = model(g, 'web')

        if args['loss']=='RN' or args['loss']=='BCERN':
            re_att = att_score(g,model)
            rn_weight = hg2rn(args,Pi,re_att,train_node,train_mask,labels)
        '''
        BCE_LOSS
        '''
        if args['loss'] == 'BCE':
            loss = loss_fcn(logits[train_mask], F.one_hot(labels[train_mask], num_classes = 2).float())
        '''
        Focal_LOSS
        '''
        if args['loss'] == 'Focal':
            loss = loss_fcn(logits[train_mask], F.one_hot(labels[train_mask], num_classes = 2)).mean()

        '''
        Focal_LOSS with ReNode
        '''
        if args['loss'] == 'RN':
            loss = loss_fcn(logits[train_mask], F.one_hot(labels[train_mask], num_classes = 2))
            loss = torch.sum(loss * rn_weight[train_mask].to(args['device'])) / loss.size(0)
        '''
        BCE_LOSS with ReNode
        '''
        if args['loss'] == 'BCERN':
            loss = loss_fcn(logits[train_mask], F.one_hot(labels[train_mask], num_classes = 2).float()).mean(dim=1)
            loss = torch.sum(loss * rn_weight[train_mask].to(args['device'])) / loss.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc, train_micro_f1, train_macro_f1 = score(logits[train_mask], labels[train_mask])
        val_loss, val_acc, val_micro_f1, val_macro_f1, val_precision, val_recall = hgt_evaluate(model, g, 'web', labels,
                                                                                                val_mask, loss_fcn)
        early_stop = stopper.step(val_loss.mean().data.item(), val_macro_f1, model)

        # print('Epoch {:d} | Train Loss {:.4f} | Train Micro f1 {:.4f} | Train Macro f1 {:.4f} | '
        #       'Val Loss {:.4f} | Val Macro Precison {:.4f} | Val Macro Recall {:.4f} | Val Micro f1 {:.4f} | Val Macro f1 {:.4f}'.format(
        #     epoch + 1, loss.item(), train_micro_f1, train_macro_f1, val_loss.mean().item(), val_precision, val_recall,
        #     val_micro_f1, val_macro_f1))

        if early_stop:
            break

    stopper.load_checkpoint(model)
    print('Seed',args['seed'])
    return hgt_test(model, g, 'web', labels, test_mask)


def hgt_train(args, G, features, labels, num_classes):
      
    train_idx, val_idx, test_idx, train_mask, val_mask, test_mask = shuffule(G,labels)

    Pi = hg2Pi(args,G)
    # rn_weight = []
    train_node = get_train_node(train_idx,labels)

    node_dict = {}
    edge_dict = {}
    for ntype in G.ntypes:
        node_dict[ntype] = len(node_dict)
    for etype in G.etypes:
        edge_dict[etype] = len(edge_dict)
        G.edges[etype].data['id'] = torch.ones(G.number_of_edges(etype), dtype=torch.long) * edge_dict[etype]

    #     Random initialize input feature
    for ntype in G.ntypes:
        G.nodes[ntype].data['inp'] = features[ntype]
        # if ntype in ['web','third','ip','dns']:
        #     G.nodes[ntype].data['inp'] = features[ntype]
        # else:
        #     emb = nn.Parameter(torch.Tensor(G.number_of_nodes(ntype), features['third'].shape[1]), requires_grad=False)
        #     nn.init.xavier_uniform_(emb)
        #     G.nodes[ntype].data['inp'] = emb

    G = G.to(args['device'])

    labels = labels.to(args['device'])
    train_mask = train_mask.to(args['device'])
    val_mask = val_mask.to(args['device'])
    test_mask = test_mask.to(args['device'])

    # model = HGT(G,
    #             node_dict, edge_dict,
    #             n_inp=features['web'].shape[1],
    #             n_hid=args['n_hid'],
    #             n_out=labels.max().item() + 1,
    #             n_layers=args['n_layers'],
    #             n_heads=4,
    #             use_norm=True).to(args['device'])

    # print('Training HGT with #param: %d' % (get_n_params(model)))
    # return hgt_batch(args,model, G,train_idx, train_mask, val_mask, test_mask, labels, Pi,train_node)

    model = HeteroRGCN(G,
                       in_size=features['web'].shape[1],
                       hidden_size=args['n_hid'],
                       out_size=labels.max().item() + 1).to(args['device'])
    print('Training RGCN with #param: %d' % (get_n_params(model)))
    return hgt_batch(args,model, G,train_idx, train_mask, val_mask, test_mask, labels, Pi,train_node)

    # model = HGT(G,
    #             node_dict, edge_dict,
    #             n_inp=features['web'].shape[1],
    #             n_hid=args['n_hid'],
    #             n_out=labels.max().item() + 1,
    #             n_layers=0,
    #             n_heads=4).to(args['device'])
    # print('Training MLP with #param: %d' % (get_n_params(model)))
    # hgt_batch(model, G,train_mask,val_mask,test_mask,labels,rn_weight)


In [ ]:
'''
HGT main.py
'''
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import csv
import numpy as np
result = []

args = {'hetero':True}
args['clip'] = 1.0
args['max_lr'] = 1e-3
args['n_hid'] = 256
args['num_epochs']=200
args['dataset'] = 'Pirate1RDE'
args['n_layers'] = 2
args['char_dim'] = 'eng_200'

'''
renode params
'''
args['pagerank_prob'] = 0.85
args['rn_base_weight'] = 0.5
args['rn_scale_weight'] = 1.0
G, features, labels, num_classes = load_data(args['char_dim'], args['dataset'])
print(G)

for i in range(100):
  args['seed'] = i
  '''
  loss
  '''
  args['loss'] = 'BCE'
  # args['loss'] = 'Focal'
  # args['loss'] = 'RN'
  # args['loss'] = 'BCERN'
  args = setup(args)

  print(args)
  # main(args)
  tmp = list(hgt_train(args,G, features, labels, num_classes))
  with open('./'+args['dataset']+'_'+args['loss']+'.csv','a+',encoding='utf-8') as csvfile:
      write = csv.writer(csvfile)
      write.writerow(tmp)
  result.append(tmp)
for item in result:
  print(item)
tmp = np.array(result)
ma = np.max(tmp,0)
me = np.mean(tmp,0)
mi = np.min(tmp,0)
print('Precision: {:.4}, +{:.4}, -{:.4}'.format(me[0],ma[0]-me[0],me[0]-mi[0]))
print('Recall: {:.4}, +{:.4}, -{:.4}'.format(me[1],ma[1]-me[1],me[1]-mi[1]))
print('F1: {:.4}, +{:.4}, -{:.4}'.format(me[2],ma[2]-me[2],me[2]-mi[2]))

In [ ]:
'''
HAN train.py
'''
import torch
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, precision_score, \
    recall_score

def han_evaluate(model, g, features, labels, mask, loss_func):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
    loss = loss_func(logits[mask], F.one_hot(labels[mask], num_classes = 2).float())
    accuracy, micro_f1, macro_f1 = score(logits[mask], labels[mask])

    return loss, accuracy, micro_f1, macro_f1

def han_test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        '''
        Plot output feature
        '''
        # feature_output1 = model.featuremap[mask].cpu()
        # plot(feature_output1,indices.cpu(),labels.cpu())

        indices = indices.cpu().numpy()
        labels = labels.cpu().numpy()
        p,r,f1,s = precision_recall_fscore_support(labels,indices)

        print(classification_report(labels, indices, digits=4))
        print(confusion_matrix(labels, indices))
    return p[1],r[1],f1[1]


def han_train(args, g, features, labels, num_classes):
    # If args['hetero'] is True, g would be a heterogeneous graph.
    # Otherwise, it will be a list of homogeneous graphs.
    
    train_idx, val_idx, test_idx, train_mask, val_mask, test_mask = shuffule(g,labels)


    if hasattr(torch, 'BoolTensor'):
        train_mask = train_mask.bool()
        val_mask = val_mask.bool()
        test_mask = test_mask.bool()

    features = features['web'].to(args['device'])

    labels = labels.to(args['device'])
    train_mask = train_mask.to(args['device'])
    val_mask = val_mask.to(args['device'])
    test_mask = test_mask.to(args['device'])

    from model import HAN
    model = HAN(meta_paths=[['wr', 'rw'],['wd', 'dw']],
                in_size=features.shape[1],
                hidden_size=args['hidden_units'],
                out_size=num_classes,
                # out_size=num_classes,
                num_heads=args['num_heads'],
                dropout=args['dropout']).to(args['device'])
    g = g.to(args['device'])


    stopper = EarlyStopping(patience=args['patience'])
    loss_fcn = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'],
                                 weight_decay=args['weight_decay'])

    for epoch in range(args['num_epochs']):
        model.train()
        logits = model(g, features)
        # loss = loss_fcn(logits[train_mask], labels[train_mask])
        loss = loss_fcn(logits[train_mask], F.one_hot(labels[train_mask], num_classes = 2).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc, train_micro_f1, train_macro_f1 = score(logits[train_mask], labels[train_mask])
        val_loss, val_acc, val_micro_f1, val_macro_f1 = han_evaluate(model, g, features, labels, val_mask, loss_fcn)
        early_stop = stopper.step(val_loss.mean().data.item(), val_macro_f1, model)

        # print('Epoch {:d} | Train Loss {:.4f} | Train Micro f1 {:.4f} | Train Macro f1 {:.4f} | '
        #       'Val Loss {:.4f} | Val Micro f1 {:.4f} | Val Macro f1 {:.4f}'.format(
        #     epoch + 1, loss.item(), train_micro_f1, train_macro_f1, val_loss.mean().item(), val_micro_f1, val_macro_f1))

        if early_stop:
            break

    stopper.load_checkpoint(model)
    print('Seed',args['seed'])
    return hgt_test(model, g, features, labels, test_mask)


In [ ]:
'''
HAN main.py
'''
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import csv
import numpy as np
result = []

args = {'hetero':True}
args['clip'] = 1.0
args['max_lr'] = 1e-3
args['n_hid'] = 256
args['num_epochs']=200
args['dataset'] = 'Pirate1RDE'
args['n_layers'] = 2
args['char_dim'] = 'eng_200'

'''
renode params
'''
args['pagerank_prob'] = 0.85
args['rn_base_weight'] = 0.5
args['rn_scale_weight'] = 1.0
G, features, labels, num_classes = load_data(args['char_dim'], args['dataset'])
print(G)

for i in range(100):
  args['seed'] = i
  '''
  loss
  '''
  args['loss'] = 'BCE'
  # args['loss'] = 'Focal'
  # args['loss'] = 'RN'
  # args['loss'] = 'BCERN'
  args = setup(args)

  print(args)
  # main(args)
  tmp = list(han_train(args,G, features, labels, num_classes))
  with open('./'+args['dataset']+'_'+args['loss']+'.csv','a+',encoding='utf-8') as csvfile:
      write = csv.writer(csvfile)
      write.writerow(tmp)
  result.append(tmp)
for item in result:
  print(item)
tmp = np.array(result)
ma = np.max(tmp,0)
me = np.mean(tmp,0)
mi = np.min(tmp,0)
print('Precision: {:.4}, +{:.4}, -{:.4}'.format(me[0],ma[0]-me[0],me[0]-mi[0]))
print('Recall: {:.4}, +{:.4}, -{:.4}'.format(me[1],ma[1]-me[1],me[1]-mi[1]))
print('F1: {:.4}, +{:.4}, -{:.4}'.format(me[2],ma[2]-me[2],me[2]-mi[2]))